### LSTM System Identification

Name: Ananda Cahyo Wibowo<br />
NRP : 07111940000128 <br />
Undergrad Thesis Title : Data Driven Gas Lift Well And Network Optimization With Neural Network Based System Identification Using Modbus Simulator

Data Preparation

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
#from datetime import datetime

#Read the csv file
df = pd.read_csv("upsample_min.csv")
df = pd.read_csv("upsample.csv")
df = pd.read_csv("upsampled_matlab_nonrevised.csv")
#df = pd.read_csv("upsampled_matlab.csv")
df = pd.read_csv("upsampled_matlab_rev.csv")
#df = pd.read_csv("upsampled_matlab_lama.csv")
df2=df.drop(df.columns[0], axis=1)
data = df['glir11'].to_numpy()

split = 0.75
epoch = 15
batchsize = 15
filename = "RNN_type3_" + f"{epoch}+{batchsize}" 

x1 = df['glir11'].to_numpy()[:int(split*len(data))]
x1 = x1.reshape(len(x1),1)
y1 = df['qt11'].to_numpy()[:int(split*len(data))]
y1 = y1.reshape(len(y1),1)

x2 = df['glir11'].to_numpy()[int(split*len(data)):]
y2 = df['qt11'].to_numpy()[int(split*len(data)):]

print(f"ukuran x train: {np.shape(x1)} ukuran y train: {np.shape(y1)}")
print(f"ukuran x test: {np.shape(x2)} ukuran y test: {np.shape(y2)}")

c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
plt.plot(np.arange(0,len(y2)),y2)
plt.plot(np.arange(0,len(y2)),x2*0.2)
plt.grid()

## Train Data

Preprocessing Data

In [ ]:
#New dataframe with only training data
df_for_training_x = x1
df_for_training_y = y1

#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(df_for_training_x)
scaler2 = scaler.fit(df_for_training_y)
df_for_training_scaled_x = scaler.transform(df_for_training_x)
df_for_training_scaled_y = scaler2.transform(df_for_training_y)

#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 5. We will make timesteps = 14 (past days data used for training). 

#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 14  # Number of past days we want to use to predict the future.

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled_x) - n_future +1):
    trainX.append(df_for_training_scaled_x[i - n_past:i, 0:df_for_training_x.shape[1]])

for i in range(n_past, len(df_for_training_scaled_y) - n_future +1):
    trainY.append(df_for_training_scaled_y[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

RNN LSTM Architecture & Training

In [ ]:
# define the Autoencoder model

model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary("")

In [ ]:
# fit the model
history = model.fit(trainX, trainY, epochs=epoch, batch_size=batchsize, validation_split=0.1, verbose=1)

In [ ]:
model.save("RNN_TFORDE1")
#model.save(filename)

In [ ]:
#from tensorflow.keras.models import load_model
#model = load_model("RNN_model_resolved")

Weights and Biasses

In [ ]:
layers = [0,1,2,4] #layer 0:lstm 1:lstm 3:dense

weights = {}
biases = {}
for layer in layers:
    weights[layer] = model.layers[layer].get_weights()[0]
    biases[layer] = model.layers[layer].get_weights()[1]

nlayer = 1
print(np.shape(weights[nlayer]))
print(weights[nlayer])

print(np.shape(biases[nlayer]))
print(biases[nlayer])

In [ ]:
xx = np.arange(0,len(history.history['loss']))

"""print(history.history['loss'])
print(history.history['val_loss'])
print(xx)"""

plt.figure(1)
plt.plot(xx,history.history['loss'], label='Training loss')
plt.plot(xx,history.history['val_loss'], label='Validation loss')
plt.title("Training Loss vs Validation Loss")
plt.xlabel("epoch")
plt.ylabel("val")
plt.legend()
plt.grid()

Predicting Values

In [ ]:
#Make prediction
#model = keras.models.load_model("RNN_Model")

n_days_for_prediction = 20
prediction = model.predict(trainX[:]) #shape = (n, 1) where n is the n_days_for_prediction

#Perform inverse transformation to rescale back to original range

prediction_copies = np.repeat(prediction, df_for_training_y.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)

print('nilai pred:',y_pred_future)

yy = scaler.inverse_transform(trainY)

x_axis = np.arange(0,y_pred_future.shape[0])

print(f"ukuran y: {np.shape(yy)} ukuran y pred: {np.shape(y_pred_future)}")

plt.figure(2)
plt.plot(x_axis,y_pred_future, label='pred')
plt.plot(x_axis,yy, label='well test')
plt.title("Comparison of TRAIN DATA: Well Production Data and LSTM Network")
plt.xlabel("time")
plt.ylabel("Oil Flow Rate Production (STB/day)")
plt.legend()
plt.grid()
plt.show()

Metric

In [ ]:
import math
from sklearn.metrics import r2_score

MSE = np.square(np.subtract(yy,y_pred_future)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:")
print(round(RMSE,2))

r2 = r2_score(yy,y_pred_future)
print("\nR2 Value:")
print(round(r2,2))

## Forecasting Value/Test Data

In [ ]:
from tensorflow.keras.models import load_model
#model = load_model("RNN_model_resolved_bagus_0.6r1") #BAGUSSSSSS
#model = load_model("RNN_model_resolved100")
model = load_model("RNN_TFORDE1")
#df2 = pd.read_csv("upsample.csv")
#df2 = df2.iloc[:,0:152]
x2 = df['glir11'].to_numpy()[int(split*len(data)):]
y2 = df['qt11'].to_numpy()[int(split*len(data)):]

#x2 = df2['glir11'].to_numpy()
#y2 = df2['qt11'].to_numpy()

#New dataframe with only testing data
x2 = x2.reshape(len(x2),1)
y2 = y2.reshape(len(y2),1)
df_for_testing_x = x2
df_for_testing_y = y2

#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(df_for_testing_x)
scaler2 = scaler.fit(df_for_testing_y)

df_for_testing_scaled_x = scaler.transform(df_for_testing_x)
df_for_testing_scaled_y = scaler2.transform(df_for_testing_y)

#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 5. We will make timesteps = 14 (past days data used for testing). 

#Empty lists to be populated using formatted testing data
testX = []
testY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 14  # Number of past days we want to use to predict the future.

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_testing_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_testing_scaled_x) - n_future +1):
    testX.append(df_for_testing_scaled_x[i - n_past:i, 0:df_for_testing_x.shape[1]])

for i in range(n_past, len(df_for_testing_scaled_y) - n_future +1):
    testY.append(df_for_testing_scaled_y[i + n_future - 1:i + n_future, 0])

testX, testY = np.array(testX), np.array(testY)

print('testX shape == {}.'.format(testX.shape))
print('testY shape == {}.'.format(testY.shape))

In [ ]:
#Make forecast
#model = keras.models.load_model("RNN_Model")

n_days_for_forecast = 20
forecast = model.predict(testX[:]) #shape = (n, 1) where n is the n_days_for_forecast

#Perform inverse transformation to rescale back to original range

forecast_copies = np.repeat(forecast, df_for_testing_y.shape[1], axis=-1)
y_fore_future = scaler.inverse_transform(forecast_copies)

#print('nilai pred:',y_fore_future)

yyy = scaler.inverse_transform(testY[:])

x_axis = np.arange(0,y_fore_future.shape[0])

print(f"ukuran y: {np.shape(yyy)} ukuran y pred: {np.shape(y_fore_future)}")

plt.figure(2)
plt.plot(x_axis,y_fore_future, label='pred')
plt.plot(x_axis,yyy, label='well test')
plt.title("Comparison of TEST DATA: Well Production Data and LSTM Network")
plt.xlabel("time")
plt.ylabel("Oil Flow Rate Production (STB/day)")
plt.legend()
plt.grid()
plt.show()

Metric

In [ ]:
import math
from sklearn.metrics import r2_score

MSE = np.square(np.subtract(yyy,y_fore_future)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:")
print(round(RMSE,2))

r2 = r2_score(yyy,y_fore_future)
print("\nR2 Value:")
print(round(r2,2))

### Test in the looping

In [ ]:
history.params

In [ ]:
"""import numpy as np
import random
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import time

#model = load_model("RNN_model_resolved100")
input = []
output = []
i = 0
n_past = 14
input_init = []
forecasting = []
while True:
    if i < n_past:
        ran = random.randint(600,2000)
        input_init.append(ran)
        input_zero = np.zeros((n_past-(i+1),))
        input_zero = input_zero.tolist()

        input_totall = input_init + input_zero
        print('VALUES',input_totall,'type',np.shape(input_totall))
        input_total = np.array(input_totall)
        input_total = np.reshape(input_total,(1,n_past,1))

        forecastt = model.predict(input_total) #shape = (n, 1) where n is the n_days_for_forecast
        forecastt = forecastt.tolist()[0][0]
        forecasting.append(forecastt)

        fig, ax_left = plt.subplots()
        ax_left.plot(list(range(len(forecasting))),forecasting,'-go', label = 'well pred')
        ax_left.set_ylabel('well pred')

        ax_right = ax_left.twinx()
        ax_right.plot(list(range(len(input_total[0,:,:]))),input_total[0,:,:], label = 'glir')
        ax_right.set_ylabel('glir')
        ax_left.legend()
        ax_right.legend()
        ax_left.grid()
        
        plt.pause(0.05)
        fig.clear()

        input_total = input_init + input_zero
        i+=1
    else:
        ran = random.randint(600,2000)
        input_totall.append(ran)
        print('VALUES',input_totall,'type',np.shape(input_totall))
        input_total = input_totall[-n_past:]
        input_total = np.array(input_total)
        input_total = np.reshape(input_total,(1,n_past,1))
        print('VALUE',input_total,'type',np.shape(input_total))

        forecastt = model.predict(input_total) #shape = (n, 1) where n is the n_days_for_forecast
        forecastt = forecastt.tolist()[0][0]
        forecasting.append(forecastt)

        #print("done")
        #print('forecasted:',forecasting)
        fig, ax_left = plt.subplots()
        ax_left.plot(list(range(len(forecasting))),forecasting,'-go', label = 'well pred')
        ax_left.set_ylabel('well pred')

        ax_right = ax_left.twinx()
        ax_right.plot(list(range(len(input_totall[:]))),input_totall[:], label = 'glir')
        ax_right.set_ylabel('glir') 
        ax_left.legend()
        ax_right.legend()
        ax_left.grid()
        
        plt.pause(0.05)
        fig.clear()
        i+=1
        #break
    #plt.show()"""